In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [48]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import np_utils
import numpy as np

# Load bộ dữ liệu MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [49]:
X_train = X_train.reshape(-1, 28 * 28)
X_test = X_test.reshape(-1, 28 * 28)

In [50]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [51]:
train_concat = np.concatenate((X_train, y_train), axis=1)
test_concat = np.concatenate((X_test, y_test), axis=1)


train_sort_idx = np.argsort(train_concat[:, -1])
train_sort = train_concat[train_sort_idx]

test_sort_idx = np.argsort(test_concat[:, -1])
test_sort = train_concat[test_sort_idx]

In [52]:
# Tiền xử lý dữ liệu
X_train = train_sort[:, :-1]
X_train = X_train.reshape(X_train.shape[0], 28, 28)
y_train = train_sort[:, -1]

X_test = test_sort[:, :-1]
X_test = X_test.reshape(X_test.shape[0], 28, 28)
y_test = test_sort[:, -1]

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [54]:
# Chuyển đổi nhãn thành one-hot encoding
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

# Xây dựng mô hình RNN
model = Sequential()
model.add(LSTM(128, input_shape=(28, 28), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(num_classes, activation='softmax'))

# Biên dịch mô hình
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

# Đánh giá mô hình trên tập kiểm tra
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
938/938 [==============================] - 15s 11ms/step - loss: 0.3951 - accuracy: 0.8735 - val_loss: 0.1235 - val_accuracy: 0.9647
Epoch 2/10
938/938 [==============================] - 11s 12ms/step - loss: 0.1096 - accuracy: 0.9669 - val_loss: 0.0894 - val_accuracy: 0.9732
Epoch 3/10
938/938 [==============================] - 9s 9ms/step - loss: 0.0738 - accuracy: 0.9778 - val_loss: 0.0493 - val_accuracy: 0.9852
Epoch 4/10
938/938 [==============================] - 10s 10ms/step - loss: 0.0562 - accuracy: 0.9831 - val_loss: 0.0421 - val_accuracy: 0.9873
Epoch 5/10
938/938 [==============================] - 11s 11ms/step - loss: 0.0460 - accuracy: 0.9858 - val_loss: 0.0328 - val_accuracy: 0.9900
Epoch 6/10
938/938 [==============================] - 9s 9ms/step - loss: 0.0371 - accuracy: 0.9885 - val_loss: 0.0285 - val_accuracy: 0.9902
Epoch 7/10
938/938 [==============================] - 10s 10ms/step - loss: 0.0322 - accuracy: 0.9901 - val_loss: 0.0287 - val_accuracy: 0.9

In [55]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import hamming_loss, jaccard_score, f1_score

# Tính toán hamming loss
y_true = y_test
y_true = np.argmax(y_true, axis = 1)

y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis = 1)

313/313 [==============================] - 2s 4ms/step


In [56]:
# CONFUSION MATRIX
cm = confusion_matrix(y_true, y_pred)
print(cm)

[[ 999    0    0    0    0    1    1    0    0    0]
 [   0 1123    1    0    0    0    0    3    0    0]
 [   0    2  986    3    0    0    0    0    0    0]
 [   0    0    1 1025    0    2    0    2    1    1]
 [   2    0    0    0  969    0    2    0    0    7]
 [   0    0    1    0    0  861    0    0    0    1]
 [   0    0    0    0    0    3 1011    0    0    0]
 [   0    1    2    0    0    0    0 1066    0    1]
 [   0    2    0    1    0    1    0    1  938    1]
 [   0    0    0    1    1    0    0    1    0  975]]


In [57]:
# Tính toán classification report
target_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
cr = classification_report(y_true, y_pred, target_names=target_names)
print(cr)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1001
           1       1.00      1.00      1.00      1127
           2       0.99      0.99      0.99       991
           3       1.00      0.99      0.99      1032
           4       1.00      0.99      0.99       980
           5       0.99      1.00      0.99       863
           6       1.00      1.00      1.00      1014
           7       0.99      1.00      0.99      1070
           8       1.00      0.99      1.00       944
           9       0.99      1.00      0.99       978

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



In [58]:
%cd /content/drive/MyDrive

from tensorflow.keras.models import load_model, save_model

save_model(model, 'model.h5')

/content/drive/MyDrive
